In [80]:
!pip install transformers

In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import torch

file_path = "train_data/dontpatronizeme_pcl.tsv"
train_filepath = "dev_data/train_semeval_parids-labels.csv"
dev_filepath = "dev_data/dev_semeval_parids-labels.csv"

df = pd.read_csv(file_path, sep='\t', header=2, names=['id', 'paragraph-id', 'keyword', 'countrycode', "paragraph", "label"])
df_filtered = df[df['paragraph'].notna()]

train_df = pd.read_csv(train_filepath)
dev_df = pd.read_csv(dev_filepath)

train_data = df_filtered[df_filtered['id'].isin(train_df['par_id'])]
dev_data = df_filtered[df_filtered['id'].isin(dev_df['par_id'])]

train_data_shuffled = shuffle(train_data, random_state=42)
dev_data_shuffled = shuffle(dev_data, random_state=42)

X_train = train_data_shuffled['paragraph'].to_numpy()
X_dev = dev_data_shuffled['paragraph'].to_numpy()

y_train = [0 if int(x) <= 1 else 1 for x in train_data_shuffled['label']]
y_dev = [0 if int(x) <= 1 else 1 for x in dev_data_shuffled['label']]

/tmp/ipykernel_3109378/1267011614.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
train_data_shuffled

,id,paragraph-id,keyword,countrycode,paragraph,label
4822,4823,@@8781228,disabled,gb,"As well as lying about helping flood victims ,...",0
8324,8325,@@14942580,vulnerable,ng,Betty Abah is passionate about this initiative...,4
2383,2384,@@7600715,in-need,sg,""" He liked to help people so I thought this co...",1
4288,4289,@@8869471,vulnerable,ng,""" The airlines are relatively small , weak and...",0
5507,5508,@@23720891,refugee,ie,"In general , people live inside their own bubb...",3
...,...,...,...,...,...,...
5846,5847,@@19919480,homeless,my,"Last year , a record 85 homes were demolished ...",0
5290,5291,@@21695353,homeless,pk,""" As a country , we can look for the missed op...",0
5491,5492,@@14069020,immigrant,hk,Any opening in which the speakers can revert t...,0
879,880,@@24188457,in-need,ke,Dennis insisted that his initiative was not in...,3


In [3]:
dev_data_shuffled

,id,paragraph-id,keyword,countrycode,paragraph,label
10056,10057,@@4197415,poor-families,ca,Darte acknowledged cutting back to the Windsor...,0
9650,9651,@@25216962,migrant,bd,UNITED States President Donald Trump has defen...,0
9119,9120,@@22467955,immigrant,ca,Saraswat said most immigrants have unique livi...,0
8504,8505,@@10179731,hopeless,pk,He said some elements were bent upon spreading...,0
1282,1283,@@3208839,refugee,ph,""" Stateless "" is the story of a forgotten grou...",4
...,...,...,...,...,...,...
9977,9978,@@13589752,homeless,in,One response to marital infidelity is divorce ...,0
9384,9385,@@1955909,homeless,tz,Various other areas have been experiencing exc...,0
9423,9424,@@18374692,hopeless,ca,Chris Selley : Maybe liquor retail in Ontario ...,0
9594,9595,@@1065878,hopeless,us,Robin Wauters is the European Editor of The Ne...,0


In [4]:
print(len(X_train))
print(len(X_dev))
print(len(y_train))
print(len(y_dev))

8375
2093
8375
2093


In [6]:
import wandb

sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'eval/f1_score',
        'goal': 'maximize'
    },
    'parameters': {

        'epochs': {
            'values' : [3, 5, 7]
        },

        'batch_size': {
            'values': [8, 16, 32]
        },

        'warmup_steps': {
            'values': [100, 500]
        },

        'learning_rate': {
            'values': [1e-5, 2e-5, 3e-5]
        }
    }
}

sweep_defaults = {
        'learning_rate': 2e-5,

        'batch_size': 16,

        'epochs': 5,

        'warmup_steps': 500
}

sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: 7z175i45
Sweep URL: https://wandb.ai/eli-carried/uncategorized/sweeps/7z175i45


In [7]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [8]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [7]:
!pip install accelerate -U
!pip install transformers[torch]

In [9]:
import wandb
import torch
from transformers import Trainer, TrainingArguments, RobertaTokenizer, RobertaForSequenceClassification
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import gc
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.cuda.empty_cache()

gc.collect()

torch.cuda.empty_cache()

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)
    
    return {"f1_score": f1, "accuracy": accuracy, "precision": precision, "recall": recall}

def train():

    wandb.init()

    total_t0 = time.time()

    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    max_length = 300
    dataset = CustomDataset(X_train, y_train, tokenizer, max_length)

    train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

    model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

    model.to(device)

    training_args = TrainingArguments(
        output_dir='./results',          
        per_device_train_batch_size=wandb.config.batch_size,  
        per_device_eval_batch_size=wandb.config.batch_size,
        num_train_epochs=wandb.config.epochs,
        learning_rate=wandb.config.learning_rate,
        warmup_steps=wandb.config.warmup_steps,
        evaluation_strategy="epoch",
        metric_for_best_model = 'eval/f1_score',
        report_to="wandb",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()

    print("Training complete!")
    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

    dev_set = CustomDataset(X_dev, y_dev, tokenizer, max_length)

    model.eval()

    results = trainer.evaluate(dev_set)

    torch.cuda.empty_cache()

    # Call the garbage collector
    gc.collect()
    
    # Ensure CUDA is aware of the freed memory
    torch.cuda.empty_cache()


In [12]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: ojrxzq5p with config:
wandb: 	batch_size: 8
wandb: 	epochs: 7
wandb: 	learning_rate: 1e-05
wandb: 	warmup_steps: 500
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'warmup_steps' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,F1 Score,Accuracy,Precision,Recall
1,0.378500,0.187417,0.586777,0.940299,0.651376,0.533835
2,0.269800,0.195380,0.564315,0.937313,0.629630,0.511278
3,0.198500,0.277517,0.615385,0.937313,0.600000,0.631579
4,0.127700,0.431161,0.584795,0.915224,0.478469,0.751880
5,0.070600,0.501787,0.591463,0.920000,0.497436,0.729323
6,0.036900,0.506262,0.588235,0.924776,0.520231,0.676692
7,0.027500,0.554268,0.581250,0.920000,0.497326,0.699248


Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-1500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-2000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-2500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-3000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-3500 already exists and is non-empty. Saving will proceed but saved res

Training complete!
Total training took 0:22:14 (h:mm:ss)


...

eval/accuracy,▁██▅▇▅
eval/f1_score,▁█▇▇█▇
eval/loss,▂▁▃▆▇█
eval/precision,▁▇▇▄▆█
eval/recall,▁▆▃█▇▃
eval/runtime,▁▁▁▁▁█
eval/samples_per_second,█████▁
eval/steps_per_second,█████▁
train/epoch,▁▁▃▃▅▆▆████
train/global_step,▁▁▃▃▅▆▆████
train/grad_norm,▁▄█▁


wandb: Agent Starting Run: nrpdf4y4 with config:
wandb: 	batch_size: 32
wandb: 	epochs: 3
wandb: 	learning_rate: 3e-05
wandb: 	warmup_steps: 500


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'warmup_steps' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss


wandb: Ctrl + C detected. Stopping sweep.


In [10]:
torch.cuda.empty_cache()

# Call the garbage collector
gc.collect()
    
# Ensure CUDA is aware of the freed memory
torch.cuda.empty_cache()

In [11]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
max_length = 300
dataset = CustomDataset(X_train, y_train, tokenizer, max_length)

train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

print(type(train_dataset))
print(train_dataset[:5])

<class 'list'>
[{'input_ids': tensor([    0, 20689, 15772,    11, 19531,  1469,  1182,    16,  1726,    30,
           10,   346,     9,  2433,  2156,   217,   474,  1272,  2156,  2196,
            8,  3766,  2156, 11902,  2156,  5263,     8, 24418,  1825,  2156,
           79,    26,  2156,     8,    22,    11,   645,     7,   912,    82,
           31, 11389,  4260,  2156,    52,   240,     7,   356,    23,   209,
          743,   479,   318,    52,   109,   295,    75,   192,   106,    50,
         1798,    59,    24,  2156,    24, 19958,   295,    75,   213,   409,
          479,    22,     2,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1